In [1]:
import ibapi
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
import tensorflow as tf
from tensorflow import keras
import datetime 
import time
from datetime import date
from sklearn.preprocessing import MinMaxScaler
import threading

df = pd.read_csv('C:\\src\\stockPredictor\\SpyHist.csv')
df['DateTime'] = pd.to_datetime(df['DateTime'])
df['Difference'] = df['Difference'].astype(float)
df['Open'] = df['Open'].astype(float)
df['Close'] = df['Close'].astype(float)
df['date'] = [d.date() for d in df['DateTime']]
df['time'] = [d.time() for d in df['DateTime']]
new_day_data = pd.read_csv('C:\\src\\stockPredictor\\dailySpy.csv')
new_day_data['DateTime'] = pd.to_datetime(new_day_data['DateTime'])
new_day_data['Difference'] = new_day_data['Difference'].astype(float)
new_day_data['Open'] = new_day_data['Open'].astype(float)
new_day_data['Close'] = new_day_data['Close'].astype(float)
new_day_data['date'] = [d.date() for d in new_day_data['DateTime']]
new_day_data['time'] = [d.time() for d in new_day_data['DateTime']]

scaler = MinMaxScaler()
df[['scalars']] = scaler.fit_transform(df[['Difference']])
new_day_data[['scalars']] = scaler.transform(new_day_data[['Difference']])

df=df.set_index('DateTime')
new_day_data=new_day_data.set_index('DateTime')

maxSteps = len(df.time.unique())



In [5]:
lastDate = df.date.unique()[-1]
todaysDate = datetime.datetime.today().date()
if lastDate < todaysDate:
    print(True)

True


In [ ]:
class IBapi(EWrapper, EClient):
	def __init__(self):
		EClient.__init__(self, self)
		self.data = [] #Initialize variable to store candle

	def historicalData(self, reqId, bar):
		self.data.append([bar.date, bar.open, bar.close])
		
def run_loop():
	app.run()

app = IBapi()
app.connect('127.0.0.1', 7496, 123)

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

time.sleep(1) #Sleep interval to allow time for connection to server

#Create contract object
contract = Contract()
contract.symbol = 'SPY'
contract.secType = 'STK'
contract.exchange = 'SMART'
contract.currency = 'USD'

#Request historical candles
app.reqHistoricalData(1, contract, '', '5 D', '5 mins', 'TRADES', 1, 2, False, [])

time.sleep(45) #sleep to allow enough time for data to be returned

df = pandas.DataFrame(app.data, columns=['DateTime', 'Open', 'Close'])
df['DateTime'] = pandas.to_datetime(df['DateTime'],unit='s').dt.tz_localize(tz='UTC')
df['DateTime'] = df['DateTime'].dt.tz_convert(tz='US/Eastern')
df['date'] = [d.date() for d in df['DateTime']]
df['time'] = [d.time() for d in df['DateTime']]
df['Difference'] = df['Close'] - df['Open']
df.to_csv('SpyHist.csv')  

print(df)


app.disconnect()